# **Homework Assignment: Understanding Splitting Criteria in CART for Regression**
---------------------

Homework6: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kos261/ML25/blob/main/Lab6/HW6.ipynb)


In this assignment, you will explore three common formulations of the splitting criterion used in **CART (Classification and Regression Trees)** for **regression problems**:

1. **Local RSS Minimization**  
2. **RSS Gain Maximization**  
3. **Total RSS Minimization**

You will investigate whether any of these criteria are equivalent, and you will design an experiment to determine which criterion is actually employed in a standard implementation such as **scikit-learn’s DecisionTreeRegressor**.



## **The Problem**

Many treatments of CART for regression describe the split selection process in different ways. Below are three frequently cited formulations. Suppose we have a dataset with features $X$ and target $y$, and we seek to choose a feature $X_j$ and a threshold $t$ to split the data into two regions $R_1(X_j, t)$ and $R_2(X_j, t)$. Denote by $\bar{y}_{R_m}$ the mean of targets within region $R_m$.

1. **Local RSS Minimization**  
   We select the feature and threshold that minimize the **sum of squared errors** in the two resulting child nodes:
   $$
   (X_j^*, t^*) = \arg\min_{X_j, t} \sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2.
   $$

2. **RSS Gain Maximization**  

   It is also a local method, looking only at a parent and two child nodes.

   We select the feature and threshold that maximize the **reduction** in RSS, computed by subtracting the RSS of the two child nodes from the RSS in the parent node:
   $$
   (X_j^*, t^*) = \arg\max_{X_j, t} \Bigl\{
   \underbrace{\sum_{i : x_i \in \text{Parent}} (y_i - \bar{y})^2}_{\text{Parent RSS}}
   \;-\;
   \underbrace{\sum_{m=1}^{2} \sum_{i : x_i \in R_m(X_j, t)} (y_i - \bar{y}_{R_m})^2}_{\text{Children RSS}}
   \Bigr\}.
   $$

3. **Total RSS Minimization**  
   For a dataset $\{(x_i, y_i)\}_{i=1}^N$ with features $X$ and target $y$, let $T$ be the current tree.

   For any split on feature $X_j$ at threshold $t$, define $T(X_j, t)$ as the new tree obtained by splitting one leaf of $T$ into two leaves $R_1(X_j, t)$ and $R_2(X_j, t)$.
   
   Let $\mathrm{Leaves}(T(X_j, t))$ be the set of all leaf indices in this new tree. For each leaf $m \in \mathrm{Leaves}(T(X_j, t))$, define:
   $$
   R_m = \{\, i \,\mid\, x_i \text{ ends in leaf } m\}.
   $$

   $R_m$ set collects all data indices $i$ whose feature vector $x_i$ is classified into the leaf node $m$ when passed through the tree $T(X_j,t)$. In other words, each leaf node $m$ in $T(X_j, t)$ corresponds to a unique path of splits, and any data point $x_i$ that follows that path is assigned to the leaf $m$; hence, it belongs to $R_m$.

   $R_m$ sets for all leafs $m \in \mathrm{Leaves}(T(X_j, t))$ define a partition of all indices.

   Then the objective of **minimizing total Residual Sum of Squares (total RSS)** is stated as:
   $$
   (X_j^*, t^*) = \arg\min_{(X_j, t)} \sum_{m \in \mathrm{Leaves}(T(X_j, t))}
   \sum_{i \in R_m} \Bigl(y_i - \overline{y}_{R_m}\Bigr)^2,
   $$
   where
   $$
   \overline{y}_{R_m} = \frac{1}{\lvert R_m \rvert}
   \sum_{i \in R_m} y_i
   $$
   is the mean response in leaf $m$.


## **Research Questions**

1. **Equivalence Analysis**  
   Determine whether the above formulations are equivalent or if they can yield different split choices. Specifically:
   - Are *local RSS minimization* and *RSS gain maximization* equivalent?
   - Does *total RSS minimization* coincide with either of these two, or is it distinct?
   
2. **Empirical Experiment**  
   Design and conduct a Python experiment to determine which of these formulations is implemented in `scikit-learn` in `DecisionTreeRegressor`. Present numerical results and plots to support your conclusion.


## **Tasks & Deliverables**

1. **Formulation Analysis**  
   - Compare *local RSS minimization*, *RSS gain maximization*, and *total RSS minimization*.
   - If you find that any pair of formulations is equivalent, provide a concise proof.  
   - If you find that they differ, construct a counterexample.

2. **Empirical Verification**  
   - Create a small artificial dataset and train a `DecisionTreeRegressor` from `scikit-learn`.
   - The dataset must be designed in a way that uniquely identifies the formulation used. Provide a short code snippet and a plot or table to support your conclusion.

3. **Report**  
   - Summarize your theoretical insights and empirical findings in a **Colab notebook**.
   - Include the relevant proofs, code, discussion, and conclusions.
   - Place the notebook in your **GitHub repository** for this course, add a link to it in your README.md and add an **“Open in Colab”** badge in the notebook so it can be launched directly.



# NOTES
We split parent into two regions $R_1$ and $R_2$

$RSS_{parent} = \sum_{y_i \in parent} (y_i - \bar{y})^2$

$RSS_{children} = \sum_{y_i \in R_{1}} (y_i - \bar{y}_{R_{1}})^2 + \sum_{y_i \in R_{2}} (y_i - \bar{y}_{R_2})^2 $

$RSS_{GAIN} = RSS_{parent} - RSS_{children}$    


**Total RSS**
Ex. Two leaves $A$ and $B$. Split $B$ into  $B_1$ and $B_2$

1) $RSS_{total} = RSS(A) + RSS(B)$
2) $RSS_{total} = RSS(A) + RSS(B_1) + RSS(B_2)$

# ANSWER 1
Maxxing $RSS_{GAIN}$ when $RSS_{parent}$ is fixed (because it doesn't depend on treshold and chosen feature) means minimazing $RSS_{children}$ which means *local RSS minimization* and *RSS gain maximization* are equivalent.


Total RSS is similar to other two when we calculate only current split and not any other splits. First two methods are **greedy** and do not track history. 

# ANSWER 2

Checking sklearn's method:
As a counterexample I created this syntethic dataset to be as weird as I could imagine in terms of amplifying differences between **greedy algorithm and backtracking methods** yet simple enough to track changes.


# EMPIRICAL EXPERIMENT

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

def compute_rss(y_subset):
    if len(y_subset) == 0:
        return 0
    return np.sum((y_subset - np.mean(y_subset)) ** 2)

def find_best_second_split(y_values):
    if len(y_values) <= 1: 
        return compute_rss(y_values)
    
    best_rss = float('inf')
    for i in range(1, len(y_values)):
        left_rss = compute_rss(y_values[:i])
        right_rss = compute_rss(y_values[i:])
        total = left_rss + right_rss
        if total < best_rss:
            best_rss = total
    
    return best_rss

# Dataset where Total RSS and Local RSS would choose different splits
X = np.array([1, 2, 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10, 11, 12]).reshape(-1, 1)
y = np.array([1, 1, 10, 10, 1 , 1 , 20, 20, 20, 1 , 20, 20])
#    splits        ^       ^      ^            ^  ^
splits =        [ 2.5,    4.5  , 6.5    ,    9.5, 10.5    ]


tree = DecisionTreeRegressor(max_depth=2, random_state=42)
tree.fit(X, y)
sklearn_first_split = tree.tree_.threshold[0]
sklearn_second_split = tree.tree_.threshold[1]
print("Sklearn's first split:", sklearn_first_split)
print("Sklearn's second split:", sklearn_second_split)

# Evaluate for local RSS (first split only)
best_local_split = None
best_local_rss = float('inf')

for split in splits:
    left_indices = [i for i in range(len(X)) if X[i] <= split]
    right_indices = [i for i in range(len(X)) if X[i] > split]
    
    left_y = [y[i] for i in left_indices]
    right_y = [y[i] for i in right_indices]
        
    left_rss = compute_rss(left_y)
    right_rss = compute_rss(right_y)
    
    total_first_level_rss = left_rss + right_rss
    
    if total_first_level_rss < best_local_rss:
        best_local_rss = total_first_level_rss
        best_local_split = split

print("Best Local RSS split (depth 1):", best_local_split)
print("Local RSS after first split:", best_local_rss)




                            ##### DEPTH 2 #####
best_global_split = None
best_global_rss = float('inf')


for split in splits:

    left_indices = [i for i in range(len(X)) if X[i] <= split]
    right_indices = [i for i in range(len(X)) if X[i] > split]
    
    left_y = [y[i] for i in left_indices]
    right_y = [y[i] for i in right_indices]

    left_second_level_rss = find_best_second_split(left_y)
    right_second_level_rss = find_best_second_split(right_y)
    total_depth2_rss = left_second_level_rss + right_second_level_rss
    
    if total_depth2_rss < best_global_rss:
        best_global_rss = total_depth2_rss
        best_global_split = split

print("Best Global RSS split (considering depth 2):", best_global_split)
print("Total RSS after depth 2 tree:", best_global_rss)

plt.figure(figsize=(10, 6))
plt.scatter(X, y)
plt.axvline(x=best_local_split, color='r', linestyle='--', label=f'Local Best Split (x={best_local_split})')
plt.axvline(x=best_global_split, color='g', linestyle='--', label=f'Global Best Split (x={best_global_split})')
plt.title('Dataset with Different Optimal Splits Based on Approach')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

# ANSWER

Results might suggest that sclearn is using local RSS maximization. I think it's weird but who am I to decide.

In [ ]:
print(tree.criterion)  # Output: 'squared_error' (which uses Local RSS)